In [1]:
from selenium import webdriver
from lxml import html,etree
import re
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
import time
import requests
import pandas as pd
from glob import glob
from pathlib import Path

In [2]:
options=webdriver.ChromeOptions()
options.add_argument('--window-size=1920,1080')
driver = webdriver.Chrome(options=options)

In [3]:
driver.get("https://www.espn.com/mlb/stats")

In [4]:
# WebDriverWait(driver,10).until(EC.presence_of_element_located((By.LINK_TEXT,'MLB')))
# clickable=driver.find_element(By.LINK_TEXT,'MLB')
# ActionChains(driver).click(clickable).perform()
# WebDriverWait(driver,10).until(EC.presence_of_element_located((By.LINK_TEXT,'Stats')))
# clickable=driver.find_element(By.LINK_TEXT,'Stats')
# ActionChains(driver).click(clickable).perform()
# WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//option[text()="Team Statistics"]/parent::select')))
# x=driver.find_element(By.XPATH,'//option[text()="Team Statistics"]/parent::select')
# select_object=Select(x)
# select_object.select_by_visible_text('Cincinnati Reds')

In [4]:
tree = html.fromstring(driver.page_source)
team_urls=tree.xpath(r'//option[starts-with(@data-url,"/mlb/team/stats/_/name/")]/@data-url')
teams = [x.split('/')[-1] for x in team_urls]

In [40]:
existing=glob(r'./mlb/*.csv')
for team_url in team_urls:
    content=requests.get('https://www.espn.com'+team_url).content
    tree=html.fromstring(content)
    player_url_ids=tree.xpath('//table[1]//a[contains(@href,"id")]/@href')
    player_ids=[x.split('/')[-1] for x in player_url_ids]
    
    player_names=tree.xpath('//table[1]//a[contains(@href,"id")]/text()')

    for player_id, player_name in zip(player_ids,player_names):
        url=rf'https://www.espn.com/mlb/player/gamelog/_/id/{player_id}'
        tree=html.fromstring(requests.get(url).content)
        years=tree.xpath('//option[@value=2022]/parent::select//option/text()')
        for year in years:
            fn=f'{team_url.split("/")[-1]}__{player_name}__{year}.csv'
            if './mlb\\' + fn in existing:
                print('skipping: ',fn)
                continue
            url=rf'https://www.espn.com/mlb/player/gamelog/_/id/{player_id}/year/{year}/category/batting'
            try:
                dfs = pd.read_html(requests.get(url).content)
            except ImportError:
                pass
                
            gamedfs = [df.iloc[:-1] for df in dfs if df.columns[0]=='Date'] #avoid non-game table and last row summary
            if len(gamedfs)>0:
                df_year=pd.concat(gamedfs) 
                df_year.to_csv(r'./mlb/'+fn)
            
            print(fn)
    

skipping:  st-louis-cardinals __Tommy Edman__2022.csv
skipping:  st-louis-cardinals __Tommy Edman__2021.csv
skipping:  st-louis-cardinals __Tommy Edman__2020.csv
skipping:  st-louis-cardinals __Tommy Edman__2019.csv
skipping:  st-louis-cardinals __Nolan Arenado__2022.csv
skipping:  st-louis-cardinals __Nolan Arenado__2021.csv
skipping:  st-louis-cardinals __Nolan Arenado__2020.csv
skipping:  st-louis-cardinals __Nolan Arenado__2019.csv
skipping:  st-louis-cardinals __Nolan Arenado__2018.csv
skipping:  st-louis-cardinals __Nolan Arenado__2017.csv
skipping:  st-louis-cardinals __Nolan Arenado__2016.csv
skipping:  st-louis-cardinals __Nolan Arenado__2015.csv
skipping:  st-louis-cardinals __Nolan Arenado__2014.csv
skipping:  st-louis-cardinals __Nolan Arenado__2013.csv
skipping:  st-louis-cardinals __Paul Goldschmidt__2022.csv
skipping:  st-louis-cardinals __Paul Goldschmidt__2021.csv
skipping:  st-louis-cardinals __Paul Goldschmidt__2020.csv
skipping:  st-louis-cardinals __Paul Goldschmid

In [34]:
print(existing[1])
print('./mlb\\' + fn in existing)

./mlb\arizona-diamondbacks __Carson Kelly__2017.csv
True


In [173]:
df=pd.concat(df_teamlist,keys=teams)
df.index=df.index.droplevel(3)
df.index=df.index.set_names(['team','player','year'])
df['Date']=pd.to_datetime(df.loc[:,'Date'].str.split(expand=True)[1]+'/'+df.index.get_level_values(level=2))
df=df.droplevel(2)

In [174]:
df

Date    OPP     Result  AB  R  H  \
team                 player                                                    
arizona-diamondbacks Christian Walker 2022-09-27   @TOR       L4-2   2  0  0   
                     Christian Walker 2022-09-22   @NYY       L5-0   2  0  0   
                     Christian Walker 2022-09-21  vsBOS       L3-2   3  0  0   
                     Christian Walker 2022-09-20  vsBOS       W7-2   4  1  2   
                     Christian Walker 2022-09-19  vsBOS  L5-3 F/10   4  0  0   
...                                          ...    ...        ...  .. .. ..   
atlanta-braves       Austin Riley     2022-05-19  vsMIL  L3-2 F/10   4  0  0   
                     Austin Riley     2022-05-18  vsMIL  W4-3 F/10   3  1  2   
                     Austin Riley     2022-05-17  vsMIL      W12-8   5  0  2   
                     Austin Riley     2022-05-16  vsSTL      W10-2   4  2  3   
                     Austin Riley     2022-05-15  vsSTL       W4-0   3  1  1   

                                       2B  3B  HR  RBI  BB  HBP  SO  SB  CS  \
team                 player                                                   
arizona-diamondbacks Christian Walker   0   0   0    0   0    0   1   0   0   
                     Christian Walker   0   0   0    0   1    0   1   0   0   
                     Christian Walker   0   0   0    0   0    0   2   0   0   
                     Christian Walker   0   0   1    1   0    0   2   0   0   
                     Christian Walker   0   0   0    0   0    0   2   0   0   
...                                    ..  ..  ..  ...  ..  ...  ..  ..  ..   
atlanta-braves       Austin Riley       0   0   0    0   0    0   3   0   0   
                     Austin Riley       0   0   1    2   0    0   1   0   0   
                     Austin Riley       0   0   0    0   0    0   1   0   0   
                     Austin Riley       1   0   0    1   0    0   0   0   0   
                     Austin Riley       0   0   1    1   0    0   1   0   0   

                                         AVG    OBP    SLG    OPS  
team                 player                                        
arizona-diamondbacks Christian Walker  0.167  0.211  0.389  0.599  
                     Christian Walker  0.188  0.235  0.438  0.673  
                     Christian Walker  0.214  0.214  0.500  0.714  
                     Christian Walker  0.273  0.273  0.636  0.909  
                     Christian Walker  0.143  0.143  0.286  0.429  
...                                      ...    ...    ...    ...  
atlanta-braves       Austin Riley      0.421  0.421  0.789  1.211  
                     Austin Riley      0.533  0.533  1.000  1.533  
                     Austin Riley      0.500  0.500  0.833  1.333  
                     Austin Riley      0.571  0.571  1.143  1.714  
                     Austin Riley      0.333  0.333  1.333  1.667  

[134 rows x 19 columns]